In [1]:
# import libraries
import datetime as dt
import pandas as pd

# never print matching warnings
import warnings
warnings.filterwarnings('ignore')

# classic division semantics in a module
from __future__ import division

#for data visualization
import plotly.offline as pyoff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# initiate the Plotly Notebook mode to use plotly offline
init_notebook_mode(connected=True)

In [3]:
# load sales data to dataframe
data = pd.read_csv('OnlineRetail.csv',header=0, encoding = 'unicode_escape')
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50,4.95,12680.0,France


In [4]:
# seeing full summary of the retail data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# there are null data in description and the CustomerID columns
# check and clean null data
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
dropna = data.dropna(subset=['CustomerID'])
dropna.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [7]:
# check & clean duplicates data
dropna.duplicated().sum()

5225

In [8]:
dropdup = dropna.drop_duplicates()
dropdup.describe()

,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


In [9]:
# the min for unit price = 0 and the min for Quantity is a negative value
# clean up negative values most likely from refunds
retail_data = dropdup[(dropdup['Quantity'] > 0) & (dropdup['UnitPrice'] > 0)]
retail_data.describe() 

,Quantity,UnitPrice,CustomerID
count,392692.000000,392692.000000,392692.000000
mean,13.119702,3.125914,15287.843865
std,180.492832,22.241836,1713.539549
min,1.000000,0.001000,12346.000000
25%,2.000000,1.250000,13955.000000
50%,6.000000,1.950000,15150.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,8142.750000,18287.000000


In [10]:
# checking data set size
retail_data.shape

(392692, 8)

In [11]:
# some research indicats that customer clusters vary by geography
# checking to see which country has the most customers
customer_country = retail_data[['Country','CustomerID']].drop_duplicates()
customer_country = customer_country.groupby(['Country'])['CustomerID'] \
                .aggregate('count').reset_index() \
                .sort_values('CustomerID', ascending=False)
customer_country.head()

,Country,CustomerID
35,United Kingdom,3920
14,Germany,94
13,France,87
30,Spain,30
3,Belgium,25


In [12]:
# restricting data to the United Kingdom only since it has the most customers
uk_data = retail_data.loc[retail_data['Country'] == 'United Kingdom']
uk_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 8:26,3.39,17850.0,United Kingdom


In [13]:
# adding revenue column
uk_data['Revenue'] = uk_data['Quantity'] * uk_data['UnitPrice']
uk_data.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
342949,566922,23344,JUMBO BAG 50'S CHRISTMAS,10,2011-09-15 14:58,2.08,16156.0,United Kingdom,20.80
355293,567917,72818,CHRISTMAS DECOUPAGE CANDLE,72,2011-09-22 17:19,0.39,14298.0,United Kingdom,28.08
371736,569224,23503,PLAYING CARDS KEEP CALM & CARRY ON,3,2011-10-02 14:00,1.25,15701.0,United Kingdom,3.75
331159,565990,22663,JUMBO BAG DOLLY GIRL DESIGN,10,2011-09-08 11:44,2.08,13577.0,United Kingdom,20.80
202527,554465,85099B,JUMBO BAG RED RETROSPOT,10,2011-05-24 12:36,2.08,13534.0,United Kingdom,20.80


In [14]:
uk_data['InvoiceDate'] = pd.to_datetime(uk_data['InvoiceDate']).dt.date
uk_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34


In [15]:
cols_of_interest = ['CustomerID', 'InvoiceDate', 'Revenue']
df = uk_data[cols_of_interest]
df.head()

,CustomerID,InvoiceDate,Revenue
0,17850.0,2010-12-01,15.30
1,17850.0,2010-12-01,20.34
2,17850.0,2010-12-01,22.00
3,17850.0,2010-12-01,20.34
4,17850.0,2010-12-01,20.34


In [16]:
from lifetimes.plotting import *
from lifetimes.utils import *
from lifetimes.estimation import *

data = summary_data_from_transaction_data(df, 'CustomerID', 'InvoiceDate', monetary_value_col='Revenue', observation_period_end='2011-12-9')
data.head()

TypeError: float() argument must be a string or a number, not 'Day'